In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
df1 = pd.read_parquet("Toma_MatrizDependiente.parquet", engine='pyarrow')
df2 = pd.read_parquet("Toma_MatrizLider.parquet", engine='pyarrow')

In [6]:
df1, df2

(        Brd  Ch LG    HG  Tstamp_us  TrgID  NHits
 0         0   0  -    57    9808.72    0.0   64.0
 1         0   1  -    53        NaN    NaN    NaN
 2         0   2  -     0        NaN    NaN    NaN
 3         0   3  -   241        NaN    NaN    NaN
 4         0   4  -    46        NaN    NaN    NaN
 ...     ...  .. ..   ...        ...    ...    ...
 639995    0  59  -  2092        NaN    NaN    NaN
 639996    0  60  -    67        NaN    NaN    NaN
 639997    0  61  -    64        NaN    NaN    NaN
 639998    0  62  -    64        NaN    NaN    NaN
 639999    0  63  -    60        NaN    NaN    NaN
 
 [640000 rows x 7 columns],
        Brd  Ch  LG   HG    Tstamp_us  TrgID  NHits
 0        -   0   0  252  4935777.888    0.0   64.0
 1        -   0   1   27          NaN    NaN    NaN
 2        -   0   2   32          NaN    NaN    NaN
 3        -   0   3   30          NaN    NaN    NaN
 4        -   0   4  324          NaN    NaN    NaN
 ...     ..  ..  ..  ...          ...    ...  

In [34]:
df1[df1["Ch"]==0]

,Brd,Ch,LG,HG,Tstamp_us,TrgID,NHits
0,0,0,-,57,9.808720e+03,0.0,64.0
64,0,0,-,49,2.742032e+04,1.0,64.0
128,0,0,-,46,5.645415e+04,2.0,64.0
192,0,0,-,60,6.683652e+04,3.0,64.0
256,0,0,-,54,1.267324e+05,4.0,64.0
...,...,...,...,...,...,...,...
639680,0,0,-,52,2.786012e+08,10012.0,64.0
639744,0,0,-,51,2.786326e+08,10013.0,64.0
639808,0,0,-,54,2.786656e+08,10014.0,64.0
639872,0,0,-,49,2.786981e+08,10015.0,64.0
